In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from datasets import load_dataset
from llama.tokenizer import Tokenizer

from mixture_of_depths.routing_transformer import ModelArgs, MoDTransformer
from mixture_of_depths.train import MoDLlamaTrainer
from mixture_of_depths.utils import set_up_env

random_state = 88
set_up_env(random_state)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


[W ProcessGroupGloo.cpp:751] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


In [2]:
dataset = load_dataset("igorktech/anekdots", split="train")
num_of_samples = 20000
dataset = dataset.select(range(num_of_samples))

In [3]:
tokenizer = Tokenizer("tokenizer.model") # follow instruction to load 
text_length_max = 64
def collate_fn(batch):
    bsz = len(batch)
    tokenized_texts = [tokenizer.encode(x['text'], bos=True, eos=True) for x in batch]
    max_text_len = max(len(t) for t in tokenized_texts)

    pad_id = tokenizer.eos_id
    tokens = torch.full((bsz, min(text_length_max, max_text_len)), pad_id, dtype=torch.long)
    for k, t in enumerate(tokenized_texts):
        tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long)[:text_length_max]
    
    return tokens[:,:-1], tokens[:,1:]

dataloader = DataLoader(
    dataset,
    batch_size=8,
    collate_fn=collate_fn,
)

### MoD

In [6]:
model_params = ModelArgs(
    dim=256,
    n_layers=4,
    n_heads=4,
    vocab_size=tokenizer.n_words,
    routing=True,
    aux_loss=True
)
model = MoDTransformer(model_params)
writer = SummaryWriter(log_dir='runs/MoD')
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

19.794432 M parameters


In [7]:
trainer = MoDLlamaTrainer(
    params=model_params,
    model=model,
    tokenizer=tokenizer,
    dataloader=dataloader
)

In [8]:
trainer.train(
    epochs=5,
    lr=3e-4,
    use_aux_loss=model_params.aux_loss,
    use_aux_predictor=model.params.aux_routing, 
    model_dir='models/MoDLlama',
    writer=writer
)

Epoch: 1:  40%|████      | 1001/2500 [02:34<03:50,  6.50it/s]

Loss at step 1000: 4.983256434202194
Causal Loss at step 1000: 0.0024631750199141608


Epoch: 1:  80%|████████  | 2001/2500 [05:09<01:18,  6.32it/s]

Loss at step 2000: 4.061281044125557
Causal Loss at step 2000: 0.0013510382889885477


Epoch: 1: 100%|██████████| 2500/2500 [06:25<00:00,  6.48it/s]


...

Loss at step 1000: 1.6624065164327622
Causal Loss at step 1000: 1.0917767894625286e-05


Epoch: 5:  80%|████████  | 2001/2500 [05:06<01:18,  6.40it/s]

Loss at step 2000: 1.6389817885160447
Causal Loss at step 2000: 1.1160827034245812e-05


Epoch: 5: 100%|██████████| 2500/2500 [06:25<00:00,  6.48it/s]


Epoch 5/5 - Loss: 1.6381528455257415


### Vanilla Llama

In [4]:
model_params = ModelArgs(
    dim=256,
    n_layers=4,
    n_heads=4,
    vocab_size=tokenizer.n_words,
    routing=False,
    aux_loss=False
)
writer = SummaryWriter(log_dir='runs/Llama')
model = MoDTransformer(model_params)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

19.794176 M parameters


In [5]:
trainer = MoDLlamaTrainer(
    params=model_params,
    model=model,
    tokenizer=tokenizer,
    dataloader=dataloader
)

In [6]:
trainer.train(
    epochs=5,
    lr=3e-4,
    use_aux_loss=model_params.aux_loss,
    use_aux_predictor=model.params.aux_routing, 
    model_dir='models/Llama',
    writer=writer
)

Epoch: 1:  40%|████      | 1001/2500 [02:23<03:28,  7.20it/s]

Loss at step 1000: 5.0363804886341095


Epoch: 1:  80%|████████  | 2001/2500 [04:49<01:14,  6.66it/s]

Loss at step 2000: 4.484910386323929


Epoch: 1: 100%|██████████| 2500/2500 [06:01<00:00,  6.91it/s]


...

Loss at step 1000: 2.656064735889435


Epoch: 5:  80%|████████  | 2001/2500 [05:07<01:14,  6.67it/s]

Loss at step 2000: 2.626405920088291


Epoch: 5: 100%|██████████| 2500/2500 [06:23<00:00,  6.52it/s]


Epoch 5/5 - Loss: 2.6183425469875337
